In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
TAVILY_KEY = os.getenv("TAVILY_API_KEY")

In [6]:
model = ChatOpenAI(model="gpt-4o-mini", api_key=OPEN_AI_API_KEY)

prompt = "What is the weather now in Hungary, Debrecen?"

response = model.invoke([HumanMessage(content=prompt)])
response.content


"I'm unable to provide real-time weather updates. To get the current weather in Debrecen, Hungary, I recommend checking a reliable weather website or app for the latest information."

# Using the Tavily tool standalone

In [7]:
tavily = TavilySearchResults(max_results=2)
search_results = tavily.invoke("What is the weather now in Hungary, Debrecen?")
search_results

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Debrecen', 'region': 'Debrecen', 'country': 'Hungary', 'lat': 47.5333, 'lon': 21.6333, 'tz_id': 'Europe/Budapest', 'localtime_epoch': 1732696125, 'localtime': '2024-11-27 09:28'}, 'current': {'last_updated_epoch': 1732695300, 'last_updated': '2024-11-27 09:15', 'temp_c': 3.4, 'temp_f': 38.1, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 197, 'wind_dir': 'SSW', 'pressure_mb': 1026.0, 'pressure_in': 30.3, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 0, 'feelslike_c': 2.7, 'feelslike_f': 36.9, 'windchill_c': 4.0, 'windchill_f': 39.3, 'heatindex_c': 4.6, 'heatindex_f': 40.2, 'dewpoint_c': 2.3, 'dewpoint_f': 36.1, 'vis_km': 4.3, 'vis_miles': 2.0, 'uv': 0.1, 'gust_mph': 4.1, 'gust_kph': 6.6}}"},
 {'url': 'https://hungaryweather.org/debrecen',
  'content': "Debrecen, Hungary Weather. 

# Using tools with a react agents

In [8]:
agent = create_react_agent(model, [tavily])

prompt = "What is OOP?"

response = agent.invoke({"messages": [HumanMessage(content=prompt)]})

response["messages"]

[HumanMessage(content='What is OOP?', additional_kwargs={}, response_metadata={}, id='3a2385f6-148f-42d1-ae98-d756c2aa73df'),
 AIMessage(content='Object-Oriented Programming (OOP) is a programming paradigm that uses "objects" to represent data and methods to manipulate that data. It is based on several key concepts:\n\n1. **Classes and Objects**: \n   - A class is a blueprint for creating objects. It defines a type of object by encapsulating data (attributes) and functions (methods) that operate on the data.\n   - An object is an instance of a class. It contains specific data and can use the methods defined by its class.\n\n2. **Encapsulation**: \n   - This principle involves bundling the data (attributes) and methods (functions) that operate on the data into a single unit or class. Encapsulation also restricts direct access to some of an object\'s components, which is a means of preventing unintended interference and misuse of the methods and data.\n\n3. **Inheritance**: \n   - Inheri

# Trying to do math without a calculator and only with an LLM

In [12]:
agent = create_react_agent(model, [tavily])

prompt = "Multiply 12311231 with 15411414223. Respond only with the number, without any extra content or punctuation"

response = agent.invoke({"messages": [HumanMessage(content=prompt)]})

response["messages"][-1].content

'18918728609628505345373'

In [14]:
result = int(response["messages"][-1].content)
correct_response = 12311231 * 15411414223
print(correct_response == result)
print(f'correct result: {correct_response}, returned result: {result}')

False
correct result: 189733480536038513, returned result: 18918728609628505345373


# Introducing our own tool to multiply numbers

In [15]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
   """Multiply two numbers."""
   return a * b

In [16]:
agent = create_react_agent(model, [tavily, multiply])

prompt = "Multiply 12311231 with 15411414223. Respond only with the number, without any extra content or punctuation"

response = agent.invoke({"messages": [HumanMessage(content=prompt)]})

response["messages"]

[HumanMessage(content='Multiply 12311231 with 15411414223. Respond only with the number, without any extra content or punctuation', additional_kwargs={}, response_metadata={}, id='f2159050-3526-4416-84c2-0680cde80304'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0w42GU81dBhpcCf0GsTwAQ14', 'function': {'arguments': '{"a":12311231,"b":15411414223}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 124, 'total_tokens': 146, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7d4cb1e7-6e39-4afd-b382-b274306aa431-0', tool_calls=[{'name': 'multiply', 'args': {'a': 12311231, 'b': 15411414223},

In [17]:
result = int(response["messages"][-1].content)
correct_response = 12311231 * 15411414223
print(correct_response == result)
print(f'correct result: {correct_response}, returned result: {result}')

True
correct result: 189733480536038513, returned result: 189733480536038513
